### Test 

In [ ]:
from addiction import *
from addiction.subject import *
from addiction.utils import *
from addiction.metrics import *
from addiction.multimatch import *
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch.utils.data import SubsetRandomSampler
from torch_geometric.nn import GCNConv, GATConv, global_mean_pool, TopKPooling
import torchvision.models as models
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from matplotlib.patches import Rectangle
from tqdm import tqdm
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split, KFold

In [ ]:
self = Trainer(train_img_loader = train_img_loader,  valid_img_loader = valid_img_loader,  test_img_loader = test_img_loader, 
               train_gaze_loader = train_gaze_loader,  valid_gaze_loader = valid_gaze_loader, test_gaze_loader = test_gaze_loader,
               batch_size = 64,  
               env = Addiction_Env(), 
               policy = Generactor().double().to(device), 
               discriminator = Discriminator().double().to(device),
               resume_from = '/root/autodl-tmp/exp/cirl/fold2_checkpoint_epoch_9_539.pth',
               traj_net = TrajectoryGAT().double().to(device),
               device = device)

In [ ]:
def cal_labels(model, data_loader, device = device):
    model = model.double().to(device)
    model.eval()
    with torch.no_grad():
        label_table = {}
        for data in data_loader:
            # data['traj_data'].x = data['traj_data'].x.double().to(device)
            probs = model(data['traj_data'].to(device)).detach().cpu()
            probs = torch.exp(probs)  
            probs = probs / probs.sum(dim=1).unsqueeze(1)   
            subjects = data['subject'].tolist()
            trail_indices = data['trail_index'].tolist()
            cues = torch.sum(torch.stack(data['cues']), dim = 0).tolist()
            label_table.update({(subj, trail_idx, cue): prob for subj, trail_idx, cue, prob in zip(subjects, trail_indices, cues, probs)})
    return label_table

In [ ]:
human_saliency = {}
def get_human_gaze(gaze_data, human_saliency):
    for data in gaze_data:
        done = (data['end'] == 1)
        if done.sum() == 0:
            continue
        subjects = data['subject']
        index = data['trail_index']
        cues = torch.sum(torch.stack(data['cues']), dim = 0)
        saliency_maps = data['saliency_map']
        for sub, idx, cue, sal in zip(subjects, index, cues, saliency_maps):
            key = (sub.item(), idx.item(), cue.item())
            # sal = normalize_map(sal.detach().cpu().numpy())
            human_saliency[key] = sal.detach().cpu().numpy()
    return human_saliency
human_saliency = get_human_gaze(train_gaze_loader, human_saliency)   
human_saliency = get_human_gaze(valid_gaze_loader, human_saliency)
human_saliency = get_human_gaze(test_gaze_loader,  human_saliency)

In [ ]:
mms = []
all_actions = []
for i_sample in range(10):
    for i_batch, batch in enumerate(self.test_img_loader):
        batch_soft_label = torch.stack([label_tables[(subj.item(), idx.item(), cue.item())]
            for subj, idx, cue in zip(batch['subject'], batch['trail_index'], torch.sum(torch.stack(batch['cues']), dim = 0))
            if (subj.item(), idx.item(), cue.item()) in label_tables
        ], dim = -1).T
        batch['soft_label'] = batch_soft_label
        self.env.set_data(batch)
        with torch.no_grad():
            self.env.reset()
            trajs = collect_trajs(self.env, self.generator.to(self.device), self.max_traj_len)
            for sub, idx, cue, traj, label in zip(batch['subject'], batch['trail_index'], torch.sum(torch.stack(batch['cues']), dim = 0), trajs, batch['label']):
                if (sub.item(), idx.item(), cue.item()) in human_saliency:
                    agent_scanpath = torch.cat([self.env.init_fix[0].unsqueeze(0), traj['actions'].to(env.device)]).cpu().numpy()
                    human_scanpath = human_scanpaths[(sub.item(), idx.item(), cue.item())]
                    agent_scanpath = {'X': agent_scanpath[:,0],'Y': agent_scanpath[:,1],'T':agent_scanpath[:,2]}
                    human_scanpath = {'X': human_scanpath[:,0],'Y': human_scanpath[:,1],'T':human_scanpath[:,2]}
                    mm_score = multimatch(agent_scanpath, human_scanpath, im_size = (1080, 1920))
                    mms.append(mm_score)